In [ ]:
from src.clients.spark_client import get_spark_session
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, trim, when, lit,  current_timestamp, try_to_timestamp


spark = get_spark_session()


base_dataframe = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('enconding', 'ISO-8859-1')
    .csv('./data/amostragem2.csv')
)



df = base_dataframe.withColumnsRenamed(
    {'Nome da Tarefa': 'nome', 
     'Data de Criação': 'data', 
     'Data de Conclusão': 'data_conclusao', 
     'Status': 'status', 
     'Tipo da Tarefa': 'tipo_tarefa', 
     'ID do Usuário': 'user_id',
     'Usuário': 'usuario'
     }
    )

df = df.withColumn('status', df['status'].cast(StringType()))
df = df.withColumn('data', df['data'].cast(StringType()))
df = df.withColumn('data_conclusao', df['data_conclusao'].cast(StringType()))


df = df.filter((df['usuario'] == 'Jeferson Klau') & (df['status'] != 3))
df = df.distinct()
df = df.withColumn('nome', trim('nome'))
df = df.withColumn('nome', when(col('nome') == '', lit(None)).otherwise(col('nome')))
df = df.dropna(subset=['nome'])

df = df.withColumn('data', when(try_to_timestamp(col('data')).isNotNull(), col('data')).otherwise(current_timestamp().cast(StringType())))



In [90]:
df.show(100)

+--------------------+------------------+-----------------+--------------------+-------------------+------+----------------+-------------+--------------------+
|                nome|       tipo_tarefa|Tipo da Tarefa ID|                data|     data_conclusao|status|Status Descrição|      usuario|             user_id|
+--------------------+------------------+-----------------+--------------------+-------------------+------+----------------+-------------+--------------------+
|         Sunt beatae|Tarefa a Ser Feita|                2| 2023-06-19 12:57:21|2024-01-30 12:57:21|     2|       Concluído|Jeferson Klau|b4853fc1f03a3a4ce...|
|        Comprar esse|    Item de Compra|                1| 2025-02-13 11:13:26|2025-06-02 20:48:40|     2|       Concluído|Jeferson Klau|b4853fc1f03a3a4ce...|
|Rerum animi ducim...|Tarefa a Ser Feita|                2| 2024-10-10 04:20:59|2024-12-13 04:20:59|     2|       Concluído|Jeferson Klau|b4853fc1f03a3a4ce...|
|     Comprar tempore|    Item de Compra

In [94]:

from pyspark.sql.functions import when, date_format, lit, col, concat, udf
import uuid

def generate_uuid():
  return str(uuid.uuid4())

uuid_udf = udf(generate_uuid, StringType())
user_id = '035c6ada-4091-703a-1837-677cad18d4a5'

df = df.withColumn('status', when(df['status'] == 1, 'TODO').otherwise('DONE'))
df = df.withColumn('user_id', when(df['user_id'] == 'b4853fc1f03a3a4cec530a98a94d89ad', user_id))
df = df.withColumn('PK', concat(lit('LIST#'), date_format(col('data'), 'yyyyMMdd')))
df = df.withColumn('SK', concat(lit('ITEM#'), uuid_udf()))  
df = df.select(df.PK, df.SK, df.user_id, df.usuario, df.nome, df.data, df.data_conclusao, df.status, df.tipo_tarefa)

df.show(50)

+-------------+--------------------+--------------------+-------------+--------------------+--------------------+-------------------+------+------------------+
|           PK|                  SK|             user_id|      usuario|                nome|                data|     data_conclusao|status|       tipo_tarefa|
+-------------+--------------------+--------------------+-------------+--------------------+--------------------+-------------------+------+------------------+
|LIST#20230619|ITEM#95d1c6f4-ca6...|035c6ada-4091-703...|Jeferson Klau|         Sunt beatae| 2023-06-19 12:57:21|2024-01-30 12:57:21|  DONE|Tarefa a Ser Feita|
|LIST#20250213|ITEM#19abc0a9-eeb...|035c6ada-4091-703...|Jeferson Klau|        Comprar esse| 2025-02-13 11:13:26|2025-06-02 20:48:40|  DONE|    Item de Compra|
|LIST#20241010|ITEM#219f4e9f-6f1...|035c6ada-4091-703...|Jeferson Klau|Rerum animi ducim...| 2024-10-10 04:20:59|2024-12-13 04:20:59|  DONE|Tarefa a Ser Feita|
|LIST#20230626|ITEM#ca92b2cd-e05...|035c

## Enviando os dados para a tabela do dynamo

In [95]:
import time
from src.clients.dynamodb_client import DynamoDBClient

TAMANHO_LOTE = 100
DELAY_EM_SEGUNDOS = 2

dynamodb = DynamoDBClient()
table = dynamodb.get_table('teste')

pandas_df = df.toPandas()
rows = pandas_df.to_dict(orient='records')

for i in range(0, len(rows), TAMANHO_LOTE):
  batch_rows = rows[i:i+TAMANHO_LOTE]

  with table.batch_writer() as batch:
    for item in batch_rows:
      
      batch.put_item(Item=item) 

  time.sleep(DELAY_EM_SEGUNDOS)

print('Todos items foram inseridos')

Todos items foram inseridos


In [ ]:
from boto3.dynamodb.conditions import Key
from pyspark.sql.types import StringType, StructField, StructType

# Query para obter as tarefas não concluídas
response = table.query(
    IndexName='user_id-PK-index',
    KeyConditionExpression=Key('user_id').eq(user_id) & Key('PK').begins_with('LIST#'),
    FilterExpression=Key('status').eq('TODO')
)

items = response['Items']

schema = StructType([
    StructField('PK', StringType(), False),
    StructField('SK', StringType(), False),
    StructField('user_id', StringType(), False),
    StructField('usuario', StringType(), False),
    StructField('nome', StringType(), False),
    StructField('data', StringType(), False),
    StructField('data_conclusao', StringType(), True),
    StructField('status', StringType(), False),
    StructField('tipo_tarefa', StringType(), False)
])

dynamodb_dataframe = spark.createDataFrame(items, schema=schema)




In [ ]:
import pyspark.sql.functions as F

def obter_abandonadas(data):
    abandonadas_df = (
        dynamodb_dataframe
        .select(
            F.col('nome'),
            F.col('data'),
            F.col('tipo_tarefa')
        
        )
        .where(
            ((F.col('tipo_tarefa') == 'Tarefa a Ser Feita') & (F.date_diff(F.lit(data), F.col('data')) >= 15)) |
            ((F.col('tipo_tarefa') == 'Item de Compra') & (F.date_diff(F.lit(data), F.col('data')) >= 30)) 
        ).orderBy('tipo_tarefa')
    )

    return abandonadas_df



In [ ]:
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta

today = date.today()
datas = [(today - relativedelta(months=i)) for i in range(5, -1, -1)]

resultados = [
    obter_abandonadas(data)
    .groupby(F.col('tipo_tarefa'))
    .count().toPandas()
    .rename(columns={'count': data.strftime('%Y-%m')}) 
    .set_index('tipo_tarefa')
    for data in datas
    ]

abandonadas_por_data = pd.concat(resultados, axis=1)
abandonadas_por_data.to_csv('abandonadas_por_data.csv', index_label='', sep=';')
    

